# Santander Dev Week 2023 (ETL com Python)


**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.

In [2]:
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'


## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.


In [3]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
df

,UserID
0,4480
1,4481
2,4482


In [4]:
user_ids = df['UserID'].tolist()
user_ids

[4480, 4481, 4482]

In [5]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 4480,
    "name": "Ruan Pablo",
    "account": {
      "id": 4748,
      "number": "300018-1",
      "agency": "0025",
      "balance": 500.65,
      "limit": 1000.0
    },
    "card": {
      "id": 4341,
      "number": "xxxx xxxx xxxx 9855",
      "limit": 1000.0
    },
    "features": [
      {
        "id": 1392,
        "icon": "string",
        "description": "string"
      }
    ],
    "news": [
      {
        "id": 8456,
        "icon": "string",
        "description": "string"
      }
    ]
  },
  {
    "id": 4481,
    "name": "Pedro Pablo",
    "account": {
      "id": 4749,
      "number": "300019-1",
      "agency": "0025",
      "balance": 500.65,
      "limit": 1000.0
    },
    "card": {
      "id": 4342,
      "number": "xxxx xxxx xxxx 9856",
      "limit": 1000.0
    },
    "features": [
      {
        "id": 1393,
        "icon": "string",
        "description": "string"
      }
    ],
    "news": [
      {
        "id": 8457,
        "icon": "string"

## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [6]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.8 MB/s eta 0:00:00


In [9]:
# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
openai_api_key = 'API-KEY'

In [10]:
import openai

openai.api_key = openai_api_key

def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em marketing bancário."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

Ruan Pablo, investir é providenciar seu futuro. Torne seu dinheiro seu empregado. Vamos começar?
Olá Pedro, invista para prosperar. Seu futuro financeiro depende das decisões de hoje. Vamos nessa!
Olá Katia Pablo! Investir é garantia de futuro próspero. Vamos começar juntos essa jornada?".


Pyterson, invista para fazer seu dinheiro crescer. Seu futuro financeiro depende disso!
Pip, investir é o caminho para multiplicar seu dinheiro. Vamos fortalecer seu futuro financeiro!
Pep, investimentos são a chave para o futuro financeiro. Cresça seu dinheiro, não apenas o guarde!

## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [11]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Ruan Pablo updated? True!
User Pedro Pablo updated? True!
User Katia Pablo updated? True!


User Pyterson updated? True!
User Pip updated? True!
User Pep updated? True!